In [ ]:
# # IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# # RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
# import kagglehub
# kagglehub.login()


In [ ]:
# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# # THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.

# danieldumi_nli_data_path = kagglehub.dataset_download('danieldumi/nli-data')

# print('Data source import complete.')


In [ ]:
import pandas as pd

import numpy as np

import string

from nltk.stem import PorterStemmer

from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords
from statsmodels.graphics.mosaicplot import mosaic

import matplotlib.pyplot as plt

import torch
from transformers import pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/

Mounted at /content/drive
MyDrive


In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/NLI/prompting/new_test_all.csv"
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLI/prompting/new_test_all.csv')

In [ ]:
test_df = pd.read_csv(file_path,index_col=0)
test_df

,premise,hypothesis,gold_label,type
9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive
9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive
9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive
9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive
9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive
...,...,...,...,...
NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive
NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive
NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive
NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive


In [ ]:
test_df['gold_label'].value_counts()

,count
gold_label,
N,879
C,768
E,720


In [ ]:
test_df['type'].value_counts()

,count
type,
type5_possessive,500
type5_again,340
type4_possessive,257
type1_again,212
type2_again,212
type2_possessive,212
type3_again,212
type3_possessive,212
type1_possessive,210


# Gemma Model

In [ ]:
from huggingface_hub import login
login(new_session=False, # Won’t request token if one is already saved on machine
write_permission=True, # Requires a token with write permission
token="", # The name of your token
add_to_git_credential=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)


In [ ]:
pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    device="cuda",
    model_kwargs={"force_download": True}
  )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
!mkdir -p /content/drive/MyDrive/Colab Notebooks/NLI/models/gemma
!cp -r /content/drive/MyDrive/Colab Notebooks/NLI/models/gemma

In [ ]:
# Prevent colab from disconnecting

from IPython.core.display import Javascript

def prevent_disconnect():
    display(Javascript('''
        function keepAlive() {
            setInterval(() => {
                console.log("Keeping session alive...");
                google.colab.kernel.execute('1+1');
            }, 60000);
        }
        keepAlive();
    '''))

prevent_disconnect()


<IPython.core.display.Javascript object>

# Llama Model

In [ ]:
!pip install modelscope

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.6 MB/s eta 0:00:00


In [ ]:
from modelscope import snapshot_download
model_id = snapshot_download("LLM-Research/Meta-Llama-3-8B-Instruct")

2025-02-06 03:05:08,307 - modelscope - INFO - Got 17 files, start to download ...
Processing 17 items:   0%|          | 0.00/17.0 [00:00<?, ?it/s]

Processing 17 items:   6%|▌         | 1.00/17.0 [00:01<00:25, 1.58s/it]

Processing 17 items:  18%|█▊        | 3.00/17.0 [00:01<00:06, 2.14it/s]

Processing 17 items:  24%|██▎       | 4.00/17.0 [00:01<00:05, 2.55it/s]

Processing 17 items:  29%|██▉       | 5.00/17.0 [00:02<00:06, 1.98it/s]

Processing 17 items:  35%|███▌      | 6.00/17.0 [00:02<00:04, 2.58it/s]

Processing 17 items:  41%|████      | 7.00/17.0 [00:03<00:05, 1.82it/s]

Processing 17 items:  47%|████▋     | 8.00/17.0 [00:04<00:05, 1.77it/s]

Processing 17 items:  53%|█████▎    | 9.00/17.0 [00:04<00:04, 1.87it/s]

Processing 17 items: 100%|██████████| 17.0/17.0 [08:03<00:00, 28.4s/it]
2025-02-06 03:13:11,326 - modelscope - INFO - Download model 'LLM-Research/Meta-Llama-3-8B-Instruct' successfully.


In [ ]:
llama_pipeline = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
terminators = [
    llama_pipeline.tokenizer.eos_token_id,
    llama_pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Gemma Prompting

In [ ]:
def classify_pair(row):
    premise = row["premise"]
    hypothesis = row["hypothesis"]
    messages = [
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: {premise}
        Hypothesis: {hypothesis}
        Output:
        '''},
    ]

    outputs = pipe(
        messages,
        max_new_tokens=1,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][-1]['content']

In [ ]:
[classify_pair(test_df.iloc[-1*i]) for i in range(10)]

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


['E', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

In [ ]:
[classify_pair(test_df.iloc[-1*i]) for i in range(100)]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


['E',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'E',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'E',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'E',
 'N',
 'N',
 'N',
 'N',
 'E',
 'N',
 'N',
 'E',
 'E',
 'E',
 'E',
 'N',
 'N',
 'E',
 'E',
 'E',
 'N',
 'E',
 'E',
 'N',
 'E',
 'N',
 'E',
 'E',
 'E',
 'E',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N']

In [ ]:
test_df.iloc[:10]

,premise,hypothesis,gold_label,type
9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive
9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive
9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive
9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive
9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive
9629.0,It's not the case that if Lily takes the train...,Lily has a friend.,E,type5_possessive
9630.0,Sam believes that if Lily takes the train to t...,Lily has a friend.,E,type5_possessive
9631.0,Is it true that if Lily takes the train to the...,Lily has a friend.,E,type5_possessive
9640.0,"If Sarah has a doctor's appointment, her husba...",Sarah has a husband.,E,type5_possessive
9641.0,It's not the case that if Sarah has a doctor's...,Sarah has a husband.,E,type5_possessive


In [ ]:
test_df['Predicted'] = test_df.apply(classify_pair, axis=1)
test_df

,premise,hypothesis,gold_label,type,Predicted
9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive,E
9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive,N
9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive,N
9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive,N
9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive,N
...,...,...,...,...,...
NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive,N
NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive,N
NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive,N
NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive,N


In [ ]:
test_df.to_csv('new_test_data_gemma_prompting_results_zero_shot.csv')

In [ ]:
test_df['gold_label'].value_counts()

,count
gold_label,
N,879
C,768
E,720


In [ ]:
test_df['Predicted'].value_counts()

,count
Predicted,
N,1965
E,387
C,15


# Llama Prompting

In [ ]:
def classify_pair_llama(row):
    premise = row["premise"]
    hypothesis = row["hypothesis"]
    messages = [
        {"role": "system", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        '''},
        {"role": "user", "content": f'''
            Premise: {premise}
            Hypothesis: {hypothesis}
            Output:
        '''},
    ]
    outputs = llama_pipeline(
        messages,
        max_new_tokens=1,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    return outputs[0]["generated_text"][-1]['content']

In [ ]:
[classify_pair_llama(test_df.iloc[-1*i]) for i in range(10)]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
test_df['Predicted_Llama'] = test_df.apply(classify_pair_llama, axis=1)
test_df

,premise,hypothesis,gold_label,type,Predicted,Predicted_Llama
9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive,E,N
9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive,N,N
9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive,N,N
9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive,N,N
9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive,N,N
...,...,...,...,...,...,...
NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive,N,N
NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive,N,N
NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive,N,N
NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive,N,N


In [ ]:
test_df.to_csv('new_test_data_Llama_prompting_results_zero_shot.csv')

# Metrics

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score

In [ ]:
def compute_metrics(predictions,labels):
    acc = np.mean(predictions == labels)
    precision = precision_score(labels, predictions,pos_label="E")
    recall = recall_score(labels, predictions,pos_label="E")
    f1 = f1_score(labels, predictions,pos_label="E")
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1_score': f1}

In [ ]:
def compute_weighted_metrics(predictions,labels):
    acc = np.mean(predictions == labels)
    precision = precision_score(labels, predictions,average="weighted")
    recall = recall_score(labels, predictions,average="weighted")
    f1 = f1_score(labels, predictions,average="weighted")
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1_score': f1}

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

def compute_metrics(predictions, labels):
    acc = np.mean(predictions == labels)
    precision = precision_score(labels, predictions, average="macro")  # Changed from binary
    recall = recall_score(labels, predictions, average="macro")  # Changed from binary
    f1 = f1_score(labels, predictions, average="macro")  # Changed from binary
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1_score': f1}

def compute_weighted_metrics(predictions, labels):
    acc = np.mean(predictions == labels)
    precision = precision_score(labels, predictions, average="weighted")  # Weighted version
    recall = recall_score(labels, predictions, average="weighted")  # Weighted version
    f1 = f1_score(labels, predictions, average="weighted")  # Weighted version
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1_score': f1}


In [ ]:
metrics = compute_metrics(test_df['gold_label'], test_df['Predicted'])
print("Macro Metrics:", metrics)

weighted_metrics = compute_weighted_metrics(test_df['gold_label'], test_df['Predicted'])
print("Weighted Metrics:", weighted_metrics)

## Gemma

In [ ]:
metrics = compute_metrics(test_df['gold_label'],test_df['Predicted'])
metrics

{'accuracy': 0.30798479087452474,
 'precision': 0.2830571158829478,
 'recall': 0.3726308591566891,
 'f1_score': 0.20328981964828138}

In [ ]:
weighted_metrics = compute_weighted_metrics(test_df['gold_label'],test_df['Predicted'])
weighted_metrics

{'accuracy': 0.30798479087452474,
 'precision': 0.6277053100816691,
 'recall': 0.30798479087452474,
 'f1_score': 0.39942824648386577}

In [ ]:
def compute_metrics_by_type(group,predicted_col):
    predictions = group[predicted_col]
    labels = group['gold_label']
    acc = np.mean(predictions == labels)
    precision = precision_score(labels, predictions,average="weighted")
    recall = recall_score(labels, predictions,average="weighted")
    f1 = f1_score(labels, predictions,average="weighted")
    return pd.Series({'accuracy': acc, 'precision': precision, 'recall': recall, 'f1_score': f1})

In [ ]:
metrics_by_type = test_df.groupby('type').apply(lambda x: compute_metrics_by_type(x, 'Predicted')).reset_index()
metrics_by_type

,type,accuracy,precision,recall,f1_score
0,type1_again,0.570755,0.356374,0.570755,0.438778
1,type1_possessive,0.300000,0.277941,0.300000,0.288550
2,type2_again,0.542453,0.485051,0.542453,0.435302
3,type2_possessive,0.462264,0.344011,0.462264,0.394465
4,type3_again,0.537736,0.675363,0.537736,0.460724
5,type3_possessive,0.273585,0.251934,0.273585,0.262314
6,type4_possessive,0.268482,0.162965,0.268482,0.195810
7,type5_again,0.076471,0.943465,0.076471,0.061406
8,type5_possessive,0.130000,0.370175,0.130000,0.157240


In [ ]:
metrics_by_type.to_csv("metrics_by_type_resultsGemma.csv", index=False)

## Llama

In [ ]:
metrics_by_type = test_df.groupby('type').apply(lambda x: compute_metrics_by_type(x, 'Predicted_Llama')).reset_index()
metrics_by_type

,type,accuracy,precision,recall,f1_score
0,type1_again,0.924528,0.952104,0.924528,0.937806
1,type1_possessive,0.752381,0.827513,0.752381,0.737675
2,type2_again,0.929245,0.929429,0.929245,0.929316
3,type2_possessive,0.867925,0.889820,0.867925,0.870679
4,type3_again,0.825472,0.804457,0.825472,0.800740
5,type3_possessive,0.750000,0.827493,0.750000,0.726497
6,type4_possessive,0.416342,0.681543,0.416342,0.302545
7,type5_again,0.038235,0.002670,0.038235,0.004951
8,type5_possessive,0.120000,0.329123,0.120000,0.140939


In [ ]:
metrics_by_type.to_csv("metrics_by_type_resultsllama_zeroShot.csv", index=False)

In [ ]:
weighted_metrics = compute_weighted_metrics(test_df['gold_label'],test_df['Predicted_Llama'])
weighted_metrics

{'accuracy': 0.527672158850866,
 'precision': 0.8543563875929843,
 'recall': 0.527672158850866,
 'f1_score': 0.6123941464606739}

# Few-Shot Learning

## Gemma

In [ ]:
def classify_pair_fewshot_gemma(row):
    premise = row["premise"]
    hypothesis = row["hypothesis"]
    messages = [
        # type 1 again
         {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Martin fostered a dog, he'll never foster a dog again.
        Hypothesis: Martin fostered a dog.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 1 possessive
        {"role": "user", "content": f'''
        Premise: If Alexander has a cheerful son-in-law, he'll trust his cheerful son-in-law.
        Hypothesis: Alexander has a cheerful son-in-law.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 2 again - negated
        {"role": "user", "content": f'''
        Premise: It's not the case that if Nadia admires this achievement by her kids, she'll never admire an achievement by her kids again.
        Hypothesis: Nadia has admired an achievement by her kids before.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 2 possessive - believes
        {"role": "user", "content": f'''
        Premise: Sam believes that if Marley has a curious friend, he gives his friend a ride home.
        Hypothesis: Marley has a friend.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 3 again - believe
        {"role": "user", "content": f'''
        Premise: Sam believes that if Alice ever drank a cocktail, she didn't drink martini again.
        Hypothesis: Alice has drunk a martini.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 3 possessive - believe
        {"role": "user", "content": f'''
        Premise: If Meredith has video games, she will play her immersive video game.
        Hypothesis: Meredith has an immersive video game.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 4 - entailment
        {"role": "user", "content": f'''
        Premise: If Matt is a scuba diver, he'll bring his wetsuit.
        Hypothesis: Matt has a wetsuit.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        # type 4 - neutral
        {"role": "user", "content": f'''
        Premise: If Roger has a student, I wonder how diligently Roger's student studies.
        Hypothesis: Roger has a student.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 5 again
        {"role": "user", "content": f'''
        Premise: If Emily passes her driving test, she'll never read a book by Dan Brown again.
        Hypothesis: Emily has read a book by Dan Brown before.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        # type 5 possessive
        {"role": "user", "content": f'''
        Premise: If Margaret's flight arrives on time, her friend will be waiting for her at the airport.
        Hypothesis: Margaret has a friend.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        {"role": "user", "content": f'''
        Premise: {premise}
        Hypothesis: {hypothesis}
        Output:
        '''},
    ]

    outputs = pipe(
        messages,
        max_new_tokens=1,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][-1]['content']

In [ ]:
[classify_pair_fewshot_gemma(test_df.iloc[-1*i]) for i in range(10)]

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

In [ ]:
test_df['Predicted - Few-Shot'] = test_df.apply(classify_pair_fewshot_gemma, axis=1)
test_df

,premise,hypothesis,gold_label,type,Predicted,Predicted_Llama,Predicted - Few-Shot
9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive,E,N,N
9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive,N,N,N
9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive,N,N,N
9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive,N,N,N
9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive,N,N,E
...,...,...,...,...,...,...,...
NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive,N,N,N
NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive,N,N,N
NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive,N,N,N
NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive,N,N,N


In [ ]:
test_df['Predicted - Few-Shot'].value_counts()

,count
Predicted - Few-Shot,
N,2043
C,184
E,140


In [ ]:
metrics_by_type = test_df.groupby('type').apply(lambda x: compute_metrics_by_type(x, 'Predicted - Few-Shot')).reset_index()
metrics_by_type

,type,accuracy,precision,recall,f1_score
0,type1_again,0.622642,0.739788,0.622642,0.544570
1,type1_possessive,0.600000,0.703279,0.600000,0.504943
2,type2_again,0.599057,0.762060,0.599057,0.484747
3,type2_possessive,0.547170,0.374533,0.547170,0.444684
4,type3_again,0.589623,0.694340,0.589623,0.489153
5,type3_possessive,0.603774,0.798742,0.603774,0.498401
6,type4_possessive,0.595331,0.566395,0.595331,0.540680
7,type5_again,0.055882,0.011064,0.055882,0.013102
8,type5_possessive,0.262000,0.852994,0.262000,0.342466


In [ ]:
# new version
metrics_by_type = test_df.groupby('type').apply(lambda x: compute_metrics_by_type(x, 'Predicted - Few-Shot')).reset_index()
metrics_by_type

,type,accuracy,precision,recall,f1_score
0,type1_again,0.622642,0.739788,0.622642,0.544570
1,type1_possessive,0.600000,0.703279,0.600000,0.504943
2,type2_again,0.599057,0.762060,0.599057,0.484747
3,type2_possessive,0.547170,0.374533,0.547170,0.444684
4,type3_again,0.589623,0.694340,0.589623,0.489153
5,type3_possessive,0.603774,0.798742,0.603774,0.498401
6,type4_possessive,0.595331,0.566395,0.595331,0.540680
7,type5_again,0.055882,0.011064,0.055882,0.013102
8,type5_possessive,0.262000,0.852994,0.262000,0.342466


In [ ]:
metrics_by_type.to_csv("gemma_metrics_few-shot-new.csv")

In [ ]:
test_df.to_csv("gemma_results_few-shot-new.csv")

In [ ]:
# new
weighted_metrics = compute_weighted_metrics(test_df['gold_label'],test_df['Predicted - Few-Shot'])
weighted_metrics

{'accuracy': 0.4465568229826785,
 'precision': 0.850465778666627,
 'recall': 0.4465568229826785,
 'f1_score': 0.5328451655580315}

In [ ]:
weighted_metrics = compute_weighted_metrics(test_df['gold_label'],test_df['Predicted - Few-Shot'])
weighted_metrics

{'accuracy': 0.4465568229826785,
 'precision': 0.850465778666627,
 'recall': 0.4465568229826785,
 'f1_score': 0.5328451655580315}

In [ ]:
weighted_metrics = compute_weighted_metrics(test_df['gold_label'],test_df['Predicted'])
weighted_metrics

{'accuracy': 0.30798479087452474,
 'precision': 0.6277053100816691,
 'recall': 0.30798479087452474,
 'f1_score': 0.39942824648386577}

In [ ]:
test_df.to_csv('gemma_results_few-shot.csv')

## Llama

In [ ]:
def classify_pair_fewshot_llama(row):
    premise = row["premise"]
    hypothesis = row["hypothesis"]
    messages = [
        {"role": "system", "content": f'''
          Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
          Output E if the hypothesis must always be true
          Output C if the hypothesis must always be false
          Output N if the hypothesis may be either true or false
          Do not output anything other than E, C, or N
        '''},
        # type 1 again
         {"role": "user", "content": f'''
            Premise: If Martin fostered a dog, he'll never foster a dog again.
            Hypothesis: Martin fostered a dog.
            Output:
        '''},
         {"role": "assistant", "content": "N"},
        # type 1 possessive
        {"role": "user", "content": f'''
            Premise: If Alexander has a cheerful son-in-law, he'll trust his cheerful son-in-law.
            Hypothesis: Alexander has a cheerful son-in-law.
            Output:
        '''},
         {"role": "assistant", "content": "N"},
        # type 2 again - negated
        {"role": "user", "content": f'''
            Premise: It's not the case that if Nadia admires this achievement by her kids, she'll never admire an achievement by her kids again.
            Hypothesis: Nadia has admired an achievement by her kids before.
            Output:
        '''},
         {"role": "assistant", "content": "N"},
        # type 2 possessive - believes
        {"role": "user", "content": f'''
            Premise: Sam believes that if Marley has a curious friend, he gives his friend a ride home.
            Hypothesis: Marley has a friend.
            Output:
        '''},
         {"role": "assistant", "content": "N"},
        # type 3 again - believe
        {"role": "user", "content": f'''
            Premise: Sam believes that if Alice ever drank a cocktail, she didn't drink martini again.
            Hypothesis: Alice has drunk a martini.
            Output:
        '''},
         {"role": "assistant", "content": "N"},
        # type 3 possessive - believe
        {"role": "user", "content": f'''
            Premise: If Meredith has video games, she will play her immersive video game.
            Hypothesis: Meredith has an immersive video game.
            Output:
        '''},
         {"role": "assistant", "content": "N"},
        # type 4 - entailment
        {"role": "user", "content": f'''
            Premise: If Matt is a scuba diver, he'll bring his wetsuit.
            Hypothesis: Matt has a wetsuit.
            Output:
        '''},
         {"role": "assistant", "content": "E"},
        # type 4 - neutral
        {"role": "user", "content": f'''
            Premise: If Roger has a student, I wonder how diligently Roger's student studies.
            Hypothesis: Roger has a student.
            Output:
        '''},
         {"role": "assistant", "content": "N"},
        # type 5 again
        {"role": "user", "content": f'''
            Premise: If Emily passes her driving test, she'll never read a book by Dan Brown again.
            Hypothesis: Emily has read a book by Dan Brown before.
            Output:
        '''},
         {"role": "assistant", "content": "E"},
        # type 5 possessive
        {"role": "user", "content": f'''
            Premise: If Margaret's flight arrives on time, her friend will be waiting for her at the airport.
            Hypothesis: Margaret has a friend.
            Output:
        '''},
         {"role": "assistant", "content": "E"},
         {"role": "user", "content": f'''
            Premise: {premise}
            Hypothesis: {hypothesis}
            Output:
        '''},
    ]
    outputs = llama_pipeline(
        messages,
        max_new_tokens=1,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][-1]['content']

In [ ]:
def classify_pair_fewshot_llama(row):
    premise = row["premise"]
    hypothesis = row["hypothesis"]
    messages = [
        # type 1 again
         {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: Sam believes that if the boss rewarded a good behavior by Nava, they'll never reward a good behavior by Nava again.
        Hypothesis: The boss rewarded a good behavior by Nava.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 1 possessive
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Alexander has a bright roommate, he'll rest with his bright roommate.
        Hypothesis: Alexander has a bright roommate..
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 2 again - negated
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: It's not the case that if Curtis reports on this issue, he'll never report on an issue again.
        Hypothesis: Curtis has reported on an issue before.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 2 possessive - believes
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: Sam believes that if Marley has a curious friend, he gives his friend a ride home.
        Hypothesis: Marley has a friend.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 3 again - believe
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: Sam believes that if Alice ever drank a cocktail, she didn't drink martini again.
        Hypothesis: Alice has drunk a martini.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 3 possessive - believe
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Meredith has video games, she will play her immersive video game.
        Hypothesis: Meredith has an immersive video game.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 4 - entailment
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Matt is a scuba diver, he'll bring his wetsuit.
        Hypothesis: Matt has a wetsuit.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        # type 4 - neutral
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Tina has a helicopter, I wonder how low Tina's helicopter hovers.
        Hypothesis: Tina has a helicopter.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 5 again
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Emily passes her driving test, she'll never read a book by Dan Brown again.
        Hypothesis: Emily has read a book by Dan Brown before.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        # type 5 possessive
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Margaret's flight arrives on time, her friend will be waiting for her at the airport.
        Hypothesis: Margaret has a friend.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: {premise}
        Hypothesis: {hypothesis}
        Output:
        '''},
    ]

    outputs = pipe(
        messages,
        max_new_tokens=1,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][-1]['content']

In [ ]:
[classify_pair_fewshot_llama(test_df.iloc[-1*i]) for i in range(10)]

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

In [ ]:
test_df

,premise,hypothesis,gold_label,type,Predicted,Predicted_Llama,Predicted - Few-Shot
9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive,E,N,N
9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive,N,N,N
9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive,N,N,N
9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive,N,N,N
9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive,N,N,E
...,...,...,...,...,...,...,...
NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive,N,N,N
NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive,N,N,N
NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive,N,N,N
NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive,N,N,N


In [ ]:
test_df['Predicted - Few-Shot Llama'] = test_df.apply(classify_pair_fewshot_llama, axis=1)
test_df

,premise,hypothesis,gold_label,type,Predicted,Predicted_Llama,Predicted - Few-Shot,Predicted - Few-Shot Llama
9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive,E,N,N,N
9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive,N,N,N,N
9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive,N,N,N,N
9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive,N,N,N,N
9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive,N,N,E,N
...,...,...,...,...,...,...,...,...
NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive,N,N,N,N
NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive,N,N,N,N
NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive,N,N,N,N
NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive,N,N,N,N


In [ ]:
metrics_by_type = test_df.groupby('type').apply(lambda x: compute_metrics_by_type(x, 'Predicted - Few-Shot Llama')).reset_index()
metrics_by_type

,type,accuracy,precision,recall,f1_score
0,type1_again,0.603774,0.364543,0.603774,0.454606
1,type1_possessive,0.600000,0.360000,0.600000,0.450000
2,type2_again,0.608491,0.762497,0.608491,0.465270
3,type2_possessive,0.608491,0.764241,0.608491,0.466619
4,type3_again,0.528302,0.283109,0.528302,0.368659
5,type3_possessive,0.603774,0.364543,0.603774,0.454606
6,type4_possessive,0.392996,0.424156,0.392996,0.250361
7,type5_again,0.047059,0.002221,0.047059,0.004242
8,type5_possessive,0.136000,0.969143,0.136000,0.163428


In [ ]:
metrics_by_type.to_csv("Llama_metrics_few-shot-new.csv")

In [ ]:
weighted_metrics = compute_weighted_metrics(test_df['gold_label'],test_df['Predicted - Few-Shot Llama'])
weighted_metrics

{'accuracy': 0.39585973806506125,
 'precision': 0.9715287059987225,
 'recall': 0.39585973806506125,
 'f1_score': 0.5399049801797813}

# **OpenAI - GPT**

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = "")

In [ ]:
import openai

def classify_pair(row):
    premise = row["premise"]
    hypothesis = row["hypothesis"]
    messages = [
        {"role": "user", "content": f'''
        Consider the truth value of the premise. If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: {premise}
        Hypothesis: {hypothesis}
        Output:
        '''},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=1,
        temperature=0.6,
        top_p=0.9
    )

    return response.choices[0].message.content.strip()

In [ ]:
[classify_pair(test_df.iloc[-1*i]) for i in range(10)]

['E', 'C', 'E', 'N', 'N', 'N', 'C', 'N', 'N', 'N']

In [ ]:
[classify_pair(test_df.iloc[-1*i]) for i in range(100)]

['N',
 'C',
 'N',
 'N',
 'N',
 'E',
 'C',
 'N',
 'N',
 'N',
 'N',
 'C',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'C',
 'C',
 'N',
 'N',
 'C',
 'C',
 'C',
 'N',
 'N',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'C',
 'N',
 'E',
 'C',
 'C',
 'N',
 'N',
 'N',
 'C',
 'C',
 'N',
 'N',
 'N',
 'N',
 'N',
 'C',
 'N',
 'N',
 'C',
 'N',
 'C',
 'N',
 'N',
 'C',
 'C',
 'N',
 'N',
 'N',
 'N',
 'C',
 'N',
 'N',
 'N',
 'N',
 'C',
 'N',
 'N',
 'N',
 'N',
 'C',
 'N',
 'N',
 'N',
 'N',
 'C',
 'C',
 'C',
 'N',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'N',
 'C',
 'N']

In [ ]:
test_df.iloc[:10]

,premise,hypothesis,gold_label,type
9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive
9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive
9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive
9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive
9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive
9629.0,It's not the case that if Lily takes the train...,Lily has a friend.,E,type5_possessive
9630.0,Sam believes that if Lily takes the train to t...,Lily has a friend.,E,type5_possessive
9631.0,Is it true that if Lily takes the train to the...,Lily has a friend.,E,type5_possessive
9640.0,"If Sarah has a doctor's appointment, her husba...",Sarah has a husband.,E,type5_possessive
9641.0,It's not the case that if Sarah has a doctor's...,Sarah has a husband.,E,type5_possessive


In [ ]:
test_df['Predicted'] = test_df.apply(classify_pair, axis=1)
test_df

,premise,hypothesis,gold_label,type,Predicted
9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive,E
9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive,N
9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive,E
9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive,E
9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive,N
...,...,...,...,...,...
NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive,N
NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive,N
NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive,N
NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive,N


In [ ]:
test_df['gold_label'].value_counts()
test_df['Predicted'].value_counts()

,count
Predicted,
N,1427
C,537
E,402
To,1


In [ ]:
metrics = compute_metrics(test_df['gold_label'],test_df['Predicted'])
metrics

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.6451204055766794,
 'precision': 0.4729105191742511,
 'recall': 0.5411455122939912,
 'f1_score': 0.4798156467788031}

In [ ]:
metrics_by_type = test_df.groupby('type').apply(lambda x: compute_metrics_by_type(x, 'Predicted')).reset_index()
metrics_by_type

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.p

,type,accuracy,precision,recall,f1_score
0,type1_again,0.924528,0.981990,0.924528,0.950534
1,type1_possessive,0.661905,0.763455,0.661905,0.629734
2,type2_again,0.886792,0.963486,0.886792,0.919857
3,type2_possessive,0.726415,0.868948,0.726415,0.781131
4,type3_again,0.797170,0.829622,0.797170,0.807004
5,type3_possessive,0.721698,0.810709,0.721698,0.677270
6,type4_possessive,0.521401,0.564039,0.521401,0.500323
7,type5_again,0.358824,0.938702,0.358824,0.489907
8,type5_possessive,0.544000,0.940009,0.544000,0.666389


In [ ]:
metrics_by_type.to_csv("Updated_OpenAI-ZeroShot_metrics_by_type_results.csv", index=False)

### **Few-shot Openai**

In [ ]:
def classify_pair_fewshot_openai(row):
    premise = row["premise"]
    hypothesis = row["hypothesis"]
    messages = [
        # type 1 again
         {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: Sam believes that if the boss rewarded a good behavior by Nava, they'll never reward a good behavior by Nava again.
        Hypothesis: The boss rewarded a good behavior by Nava.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 1 possessive
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Alexander has a bright roommate, he'll rest with his bright roommate.
        Hypothesis: Alexander has a bright roommate..
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 2 again - negated
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: It's not the case that if Curtis reports on this issue, he'll never report on an issue again.
        Hypothesis: Curtis has reported on an issue before.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 2 possessive - believes
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: Sam believes that if Marley has a curious friend, he gives his friend a ride home.
        Hypothesis: Marley has a friend.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 3 again - believe
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: Sam believes that if Alice ever drank a cocktail, she didn't drink martini again.
        Hypothesis: Alice has drunk a martini.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 3 possessive - believe
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Meredith has video games, she will play her immersive video game.
        Hypothesis: Meredith has an immersive video game.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 4 - entailment
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Matt is a scuba diver, he'll bring his wetsuit.
        Hypothesis: Matt has a wetsuit.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        # type 4 - neutral
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Tina has a helicopter, I wonder how low Tina's helicopter hovers.
        Hypothesis: Tina has a helicopter.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 5 again
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Emily passes her driving test, she'll never read a book by Dan Brown again.
        Hypothesis: Emily has read a book by Dan Brown before.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        # type 5 possessive
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Margaret's flight arrives on time, her friend will be waiting for her at the airport.
        Hypothesis: Margaret has a friend.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: {premise}
        Hypothesis: {hypothesis}
        Output:
        '''},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=1,
        temperature=0.6,
        top_p=0.9
    )

    return response.choices[0].message.content.strip()

In [ ]:
[classify_pair_fewshot_openai(test_df.iloc[-1*i]) for i in range(10)]

['E', 'C', 'E', 'N', 'N', 'E', 'C', 'N', 'N', 'N']

In [ ]:
test_df['Predicted - Few-Shot openai'] = test_df.apply(classify_pair_fewshot_openai, axis=1)
test_df

In [ ]:
metrics_by_type = test_df.groupby('type').apply(lambda x: compute_metrics_by_type(x, 'Predicted - Few-Shot openai')).reset_index()
metrics_by_type

,type,accuracy,precision,recall,f1_score
0,type1_again,0.599057,0.363409,0.599057,0.452385
1,type1_possessive,0.600000,0.360000,0.600000,0.450000
2,type2_again,0.603774,0.364543,0.603774,0.454606
3,type2_possessive,0.613208,0.764241,0.613208,0.475725
4,type3_again,0.533019,0.365090,0.533019,0.379875
5,type3_possessive,0.608491,0.762497,0.608491,0.465270
6,type4_possessive,0.392996,0.456218,0.392996,0.245460
7,type5_again,0.047059,0.002215,0.047059,0.004230
8,type5_possessive,0.136000,0.969143,0.136000,0.165006


In [ ]:
metrics_by_type.to_csv("openai_metrics_few-shot-new.csv")

In [ ]:
weighted_metrics = compute_weighted_metrics(test_df['gold_label'],test_df['Predicted - Few-Shot openai'])
weighted_metrics